# Web Scraper for *everynoise.com*

In [1]:
#adapted from https://github.com/aweitz/EveryNoise/blob/master/scrapGenres.ipynb
from bs4 import BeautifulSoup
import requests, sys
import pandas as pd
import os
import re
import time

# Parallel and Multiprocessing Packages
from threading import get_ident
from multiprocessing import cpu_count
import asyncio
from concurrent.futures import ThreadPoolExecutor as PoolExecuter

from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend, Parallel, delayed, wrap_non_picklable_objects

# Custom Scripts
direc = sys.path.insert(1, '..')
from misc_tools.data_tools import chunkIt 

In [2]:
r = requests.get("http://everynoise.com/everyschool.cgi?scope=US")

## Scraper for *everyschool*

Goes to the website and loops through all the schools that are in the USA. Gets the top genres of each school. 

In [3]:
# Link here and read into Python, or just an html link as markdown.
# ?root=The%20University%20of%20Texas%20at%20Austin&
norm = re.compile('[+»&\s-]')
soup = BeautifulSoup(r.text,"html.parser")
allSchools = soup.find_all("tr", "datarow")
## Cycle through genres and go to that genres page
schools = schools[:-1]
# popular_genres = []
# playlists = []
# schCnt = 0
for schoolRow in allSchools[629:]:
    print("Pulling school #" + str(schCnt), end = '\t')
    sys.stdout.flush()
    school = schoolRow.text.split('USA')[1].strip()
    print(school)
    schools.append(school)
    schoolPage = "http://everynoise.com/everyschool.cgi?root=" + school.replace(' ','%20') + "&scope=US"
    ## Pull artists from genre page
    r2 = requests.get(schoolPage)
    soup2 = BeautifulSoup(r2.text,"html.parser")
    schools_genre_div = soup2.find_all("div","note")
    inner = soup2.find_all("div","note")
    for d in inner:
        genres = d.text.split('\n')[1:-1]
        genres = [re.sub(norm,'',g) for g in genres]
    popular_genres.append(genres)
    spotify = soup2.find("iframe")
    playlists.append(spotify['src'])
    schCnt = schCnt+1

NameError: name 'schools' is not defined

## Constructs the DataFrame for the USA Universities

In [ ]:
print(len(schools),len(popular_genres),len(playlists))
df = pd.DataFrame({'SCHOOL':schools, 'GENRES':popular_genres, 'PLAYLIST_LINK':playlists})
df.to_csv('school_genres.csv')
df

## Scraper for *genremap* 
Goes to the website and enters each genre link and scraps the html. Grabs the related genres, opposite genres, and uses the font size for each as a  indicator of edge 'weight'. Also gets the Spotify playlist links.

In [4]:
r = requests.get("http://everynoise.com/engenremap.html")

## Parallel Scraper Function

In [38]:
@wrap_non_picklable_objects
def genre_scraper(subgenre_subset):
    # Metadata
    subgenre_list_artist = []
    subgenre_list_opp = []
    subgenre_list_sim = []
    subgenre_list = []
    playlists = []
    sim_weights = []
    opp_weights = []
    artist_weights = []

    split_genrecount = 0
    
    # Looping over subgenres to get metadata
    for genreDiv in subgenre_subset:
        subgenre = re.sub("[:'+»&\s-]", '', genreDiv)
        
        # Keeps ' and & for the names
        subgenre_name = re.sub("[:+»\s]", ' ', genreDiv).strip()
        subgenre_list.append(subgenre_name)
        
        # Printing Genre Name
        sys.stdout.flush()
        print('Pulling genre # {} - Subgenre: {}\n'.format(split_genrecount, subgenre_name))
        sys.stdout.flush()
        
        # Fetch the subgenre
        subgenre_page = 'http://everynoise.com/engenremap-' + subgenre + '.html'
        r2 = requests.get(subgenre_page)
        soup2 = BeautifulSoup(r2.text, 'html.parser')

        
        spotify_link = soup2.find_all('a', text = 'playlist')
        playlists.append(spotify_link[0]['href'])
        allArtistDivs = set(soup2.find_all("div", "genre scanme"))
        allGenresRelated = set(soup2.find_all("div", "genre"))
        allGenresRelated = allGenresRelated - allArtistDivs
        
        # Temp Variables
        artist_list = []
        art_w = []
        
        # Get artists associated with the subgenre
        for artist in allArtistDivs:
            weight = artist['style'].split()[-1].replace('%', '')
            artistName = artist.text.strip().replace("»", "")
            if not(artistName.isspace()):
                art_w.append(weight)
                artist_list.append(artistName)
        artist_weights.append(art_w)
        subgenre_list_artist.append(artist_list)
        
        # Temp variables
        weights_sim = []
        weights_opp = []
        opp_genres = []
        sim_genres = []
        
        # Get similar / opposite genres
        for other_genre in allGenresRelated:
            weight = other_genre['style'].split()[-1].replace('%', '')
            if 'nearby' in other_genre['id']:
                g = other_genre.text.strip().replace("»", '')
                if g != subgenre_name:
                    weights_sim.append(weight)
                    sim_genres.append(g)
            elif 'mirror' in other_genre['id']:
                weights_opp.append(weight)
                opp_genres.append(other_genre.text.strip().replace("»", ''))
                
        subgenre_list_sim.append(sim_genres)
        subgenre_list_opp.append(opp_genres)
        sim_weights.append(weights_sim)
        opp_weights.append(weights_opp)
        split_genrecount += 1
    return [subgenre_list, playlists, subgenre_list_sim, sim_weights, subgenre_list_opp, opp_weights, subgenre_list_artist, artist_weights]

In [39]:
def parallel_wrapper(allGenreDivs, n_jobs = cpu_count()):
    subgenre_subsets = chunkIt(allGenreDivs, n_jobs)
    results = Parallel(n_jobs = n_jobs)(delayed(genre_scraper)(subset) for subset in subgenre_subsets)
    res = list(map(list, zip(*results)))
    res_2 = []
    for i in res:
        q = []
        for j in i:
            q += j
        res_2.append(q)
    return(res_2)

In [40]:
soup = BeautifulSoup(r.text,"html.parser")
allGenreDivs = soup.find_all("div", "genre scanme")
agd = []
for i in allGenreDivs:
    agd.append(i.text)
t1 = time.time()
music_data = parallel_wrapper(agd)
t2 = time.time()
print('Time Taken to Gather Data: {:.02f}s'.format(t2-t1))

Time Taken to Gather Data: 161.10s


In [ ]:
soup = BeautifulSoup(r.text,"html.parser")
allGenreDivs = soup.find_all("div", "genre scanme")

# Cycle through genres and go to that genres page
genreListArtist = []
genreListOpp = []
genreListSim = []
genreList = []
playlists = []
simWeights = []
oppWeights = []
artistWeights = []
artistList = []
art_w = []
genreCnt = 0
for genreDiv in allGenreDivs:
    print("Pulling genre #" + str(genreCnt))
    sys.stdout.flush()
    genre = (genreDiv.text)
    genre = re.sub("[:'+»&\s-]", '', genre)    
    genreList.append(genre)
    genrePage = "http://everynoise.com/engenremap-" + genre + ".html"

    ## Pull artists from genre page
    r2 = requests.get(genrePage)
    soup2 = BeautifulSoup(r2.text,"html.parser")
    spotify_link = soup2.find_all("a", text = 'playlist')
    playlists.append(spotify_link[0]['href'])
    allArtistDivs = set(soup2.find_all("div", "genre scanme"))
    allGenresRelated = set(soup2.find_all("div", "genre"))
    allGenresRelated = allGenresRelated - allArtistDivs

    
    for artist in allArtistDivs:
        weight = artist['style'].split()[-1].replace('%', '')
        artistName = artist.text.strip().replace("»", "")
        if not(artistName.isspace()):   
            art_w.append(weight)
            artistList.append(artistName) 
    artistWeights.append(art_w)
    genreListArtist.append(artistList)
    
    simGenres = []
    oppGenres = []
    weights_sim = []
    weights_opp = []
    
    for other_genre in allGenresRelated:
        weight = other_genre['style'].split()[-1].replace('%', '')
        if 'nearby' in other_genre['id']:
            weights_sim.append(weight)
            simGenres.append(other_genre.text.strip().replace("»", ""))
        elif 'mirror' in other_genre['id']:
            weights_opp.append(weight)
            oppGenres.append(other_genre.text.strip().replace("»", "").replace(' ', ''))
            print(other_genre.text.strip().replace("»", ""))
            
    genreListSim.append(simGenres)
    genreListOpp.append(oppGenres)
    simWeights.append(weights_sim)
    oppWeights.append(weights_opp)
    genreCnt = genreCnt+1

## Constructs the Dataframe for All Genres and Converts it to a CSV

In [41]:
# [subgenre_list, playlists, subgenre_list_sim, sim_weights, subgenre_list_opp, opp_weights, subgenre_list_artist, artist_weights]
df = pd.DataFrame({"GENRE":music_data[0], 
                   "SIM_GENRES":music_data[2],
                   "SIM_WEIGHTS":music_data[3],
                   "OPP_GENRES":music_data[4],
                   "OPP_WEIGHTS":music_data[5],
                   "REL_ARTISTS":music_data[6], 
                   "ARTIST_WEIGHTS":music_data[7],
                   "SPOTIFY_URL":music_data[1]})
df.to_csv('all_genres1.csv')
df

,GENRE,SIM_GENRES,SIM_WEIGHTS,OPP_GENRES,OPP_WEIGHTS,REL_ARTISTS,ARTIST_WEIGHTS,SPOTIFY_URL
0,classical contralto,"[vintage classical singing, classical mezzo-so...","[107, 102, 101, 102, 103, 100, 100, 108]","[hard house, tekno, breakbeat, nu skool breaks...","[126, 115, 140, 110, 105, 130, 157, 116, 100, ...","[Marijana Mijanovic, Bernadette Manca Di Nissa...","[123, 107, 102, 111, 132, 123, 109, 105, 122, ...",https://open.spotify.com/user/thesoundsofspoti...
1,piano blues,"[stride, memphis blues, texas blues, flamenco,...","[106, 100, 102, 101, 108, 100, 118, 100, 102, ...","[industrial metal, darksynth, neo-industrial r...","[151, 106, 160, 101, 136, 140, 130, 123, 118, ...","[Leroy Carr, Roomful Of Blues, Willie Mabon, O...","[103, 103, 103, 106, 105, 101, 100, 100, 101, ...",https://open.spotify.com/user/thesoundsofspoti...
2,poezja spiewana,"[folk, michigan folk, nueva trova chilena, dis...","[101, 103, 102, 102, 101, 100, 100, 103, 100, ...","[future funk, industrial rock, dance-punk, cyb...","[101, 138, 119, 108, 160, 115, 106, 107, 111, ...","[Anna Czartoryska, Paulina Bisztyga, PÃ³ki Co,...","[101, 101, 100, 103, 120, 107, 101, 101, 123, ...",https://open.spotify.com/user/thesoundsofspoti...
3,sung poetry,"[disney espanol, dutch disney, michigan folk, ...","[100, 101, 101, 129, 100, 103, 100, 100, 101, ...","[industrial hip hop, dance-punk, rebel blues, ...","[108, 129, 101, 124, 111, 113, 160, 105, 126, ...","[Andrzej Rosiewicz, Jan Kondrak, Swietliki, Al...","[107, 101, 114, 113, 117, 104, 104, 113, 115, ...",https://open.spotify.com/user/thesoundsofspoti...
4,euphoric hardstyle,"[gabba, hardstyle, frenchcore, j-core, rawstyl...","[103, 127, 100, 101, 131, 100]","[northumbrian folk, jazz vibraphone, hard bop,...","[125, 105, 104, 100, 109, 127, 160, 128, 123, ...","[D.E.Q, Nick Novity, Proto Bytez, Mark With a ...","[103, 103, 104, 117, 101, 100, 124, 100, 102, ...",https://open.spotify.com/user/thesoundsofspoti...
...,...,...,...,...,...,...,...,...
3933,norwegian techno,"[irish techno, techno, leipzig electronic, flo...","[102, 105, 100, 105, 102, 104, 103, 106, 105, ...","[sarangi, hindustani vocal, classic tunisian p...","[105, 102, 102, 126, 160, 114, 132, 102, 103, ...","[Kim DÃ¼rbeck, Ismistik, DJ Ibon, +Plattform, ...","[101, 105, 100, 103, 100, 103, 101, 126, 110, ...",https://open.spotify.com/user/thesoundsofspoti...
3934,footwork,"[minimal dubstep, fluxwork, vapor house, outsi...","[102, 117, 115, 107, 101, 102, 100, 103, 100, ...","[scottish fiddle, starogradska, shantykoren, d...","[104, 114, 151, 107, 112, 113, 106, 101, 117, ...","[Paveun, Fracture, DJ Diamond, Actress, DJ Man...","[100, 120, 104, 130, 108, 112, 100, 114, 103, ...",https://open.spotify.com/user/thesoundsofspoti...
3935,experimental poetry,[early avant garde],[100],"[anime latino, nightrun, anime, j-core, vocalo...","[133, 123, 111, 113, 144, 126, 100, 102, 160, ...","[Robert Creeley, Marcel Broodthaers, Ivor Cutl...","[105, 100, 160, 100, 100, 100, 102, 100, 112, ...",https://open.spotify.com/user/thesoundsofspoti...
3936,grimewave,"[nz electronic, vogue, glitch hop, acid idm, e...","[104, 111, 103, 100, 114, 107, 106, 106, 103, ...","[brill building pop, siberian folk, canzone na...","[100, 117, 103, 117, 126, 160, 107, 119, 116, ...","[Rabit, Prettybwoy, She's Drunk, Soda Plains, ...","[108, 101, 100, 104, 103, 100, 101, 109, 101, ...",https://open.spotify.com/user/thesoundsofspoti...
